In [3]:
!pip install transliterate

  Using cached transliterate-1.10.2-py2.py3-none-any.whl (45 kB)


In [5]:
!pip install pymorphy2

  Using cached pymorphy2-0.8-py2.py3-none-any.whl (46 kB)
Processing c:\users\vital\appdata\local\pip\cache\wheels\9b\04\dd\7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e\docopt-0.6.2-py2.py3-none-any.whl
  Using cached pymorphy2_dicts-2.4.393442.3710985-py2.py3-none-any.whl (7.1 MB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)


In [1]:
import transliterate
from transliterate import translit, get_available_language_codes
import pymorphy2
import os
import re
import string
from collections import Counter
from nltk.corpus import stopwords

# Т.

## Скачивание файлов

In [2]:
folder = []
for i in os.walk('childes_Tosia'):
    folder.append(i)

Выводим список всех файлов и папок 

In [3]:
#folder

In [4]:
filenames = []
for address, dirs, files in folder:
    for file in files:
        filenames.append(file)

Выводим список всех файлов, содержащих диалоги

In [5]:
#filenames

In [6]:
paths = []  # список готовых директорий к файлам с диалогами
for filename in filenames:
    if filename.startswith('TD'):
        directory = 'childes_Tosia/malenkaya/'
    else:
        directory = 'childes_Tosia/'
    path = directory + filename
    paths.append(path)
    #print(path)

#print(len(paths))

## Разделение реплик на то, что ребенок слышит, и то, что он говорит

In [7]:
morph = pymorphy2.MorphAnalyzer()

dict_words = []

textChild = ''
textParents = ''

for filename in paths:
    with open(filename, 'r', encoding='utf-8') as f:
        file = f.readlines()
        for line in file:
            line = translit(line, 'ru')
            line = re.sub('йо', 'ё', line)
            line = re.sub('шч', 'щ', line)
            if line.startswith('*ЦХИ'):  # выбираем только реплики ребенка 
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textChild += line.lower()
            elif line.startswith('*'):
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textParents += line.lower()
        
with open('TosiaSays.txt', 'w+', encoding='utf-8') as f:
    f.write(textChild)
with open('TosiaHears.txt', 'w+', encoding='utf-8') as f:
    f.write(textParents)

In [8]:
def WordsCounting(filepath):
    with open(filepath, 'r+', encoding='utf-8') as f:
        text = f.readlines()
        # список из всех слов, которые говорит или слышит ребенок
        words = []
        for line in text:
            if ' [: ' in line:
                line = line.replace(' [:', 'ERR')
            for word in line.lower().translate(str.maketrans(dict.fromkeys(string.punctuation))).split():
                if not word.endswith('err'):
                    words.append(word)

    # список из словарей вида слово:часть речи
    dict_words = [] 
    for word in words:
        pos = morph.parse(word)[0].tag.POS
        d = {}
        d[word] = pos
        dict_words.append(d)

    pos = []   
    
    # топ-10 слов и топ-10 существительных, которые слышит или говорит ребенок
    all_words = []
    all_nouns = []
    forbidden = ['цхи', 'ххх', 'мот', 'фат', 'æто',
                 'лаугхинга', 'чико', 'нана', 'мотхер', 'бро',
                 'мота', 'xxx', 'сис', '\x15\x15', 'ала',
                'деу', 'клякляу', 'цхилд', 'нанна', 'сцреаминг']
    for d in dict_words:
        for key, value in d.items():
            key = morph.parse(key)[0].normal_form
            if key not in forbidden:
                all_words.append(key)
                if value != None:
                    pos.append(value)
            if value == 'NOUN' and key not in forbidden:
                all_nouns.append(key)

    # топ частей речи, которые слышит или говорит ребенок
    top_pos = Counter(pos)
    print('Распределение по частям речи ', top_pos)
                
    # общая размерность корпуса вместе со словами, у которых тэг части речи None
    print('Всего слов в этой части корпуса (вместе с неопределенной частью речи)', len(all_words))
    print('Всего слов в этой части корпуса (часть речи определена)', len(pos))
                
    sw = stopwords.words('russian')
    
    filtered_words = [w for w in all_words if w not in sw]
    filtered_nouns = [w for w in all_nouns if w not in sw]
    
    top_words = Counter(filtered_words)
    top_words = dict(top_words)
    top_words = {c: top_words[c] for c in top_words if top_words[c] > 2 and len(c) > 2}
    top_words = (sorted(top_words.items(), key=lambda x: x[1], reverse=True))[:10]
    
    top_nouns = Counter(filtered_nouns)
    top_nouns = dict(top_nouns)
    top_nouns = {c: top_nouns[c] for c in top_nouns if top_nouns[c] > 2 and len(c) > 2}
    top_nouns = (sorted(top_nouns.items(), key=lambda x: x[1], reverse=True))[:11]
    
    print('Топ-10 всех слов', top_words)
    print('Топ-10 всех существительных', top_nouns)

### Что Т. говорит

In [9]:
WordsCounting('TosiaSays.txt')

Распределение по частям речи  Counter({'NOUN': 2319, 'NPRO': 1327, 'VERB': 1259, 'PRCL': 1234, 'ADVB': 613, 'CONJ': 559, 'ADJF': 475, 'PRED': 464, 'INTJ': 343, 'PREP': 294, 'INFN': 209, 'NUMR': 75, 'ADJS': 56, 'GRND': 40, 'COMP': 27, 'PRTS': 7})
Всего слов в этой части корпуса (вместе с неопределенной частью речи) 10135
Всего слов в этой части корпуса (часть речи определена) 9301
Топ-10 всех слов [('xочить', 102), ('няня', 100), ('ещё', 88), ('ама', 83), ('смотреть', 77), ('видеть', 76), ('мочь', 62), ('большой', 62), ('варя', 54), ('тося', 52)]
Топ-10 всех существительных [('няня', 100), ('ама', 83), ('варя', 54), ('тося', 52), ('мама', 36), ('стасик', 33), ('метр', 25), ('папа', 24), ('кошка', 19), ('ёлочка', 19), ('ромашка', 18)]


### Что Т. слышит

In [10]:
WordsCounting('TosiaHears.txt')

Распределение по частям речи  Counter({'NOUN': 7022, 'VERB': 4495, 'CONJ': 3016, 'NPRO': 2884, 'PRCL': 2788, 'ADVB': 2129, 'PREP': 1725, 'ADJF': 1640, 'INFN': 902, 'INTJ': 662, 'PRED': 431, 'ADJS': 405, 'GRND': 218, 'NUMR': 168, 'COMP': 93, 'PRTS': 39, 'PRTF': 18})
Всего слов в этой части корпуса (вместе с неопределенной частью речи) 29665
Всего слов в этой части корпуса (часть речи определена) 28635
Топ-10 всех слов [('тося', 454), ('давать', 301), ('ещё', 173), ('смотреть', 163), ('мочь', 139), ('сказать', 135), ('весь', 135), ('тосй', 109), ('пойти', 108), ('дать', 105)]
Топ-10 всех существительных [('тося', 454), ('тосй', 109), ('мама', 102), ('тоня', 63), ('варя', 62), ('свет', 59), ('молодец', 54), ('ромашка', 48), ('девочка', 42), ('нога', 41), ('ножка', 40)]


# Я.

## Скачивание файлов

In [12]:
folder = []
for i in os.walk('childes_Yasha'):
    folder.append(i)

Выводим список всех файлов и папок

In [13]:
#folder

In [14]:
filenames = []
for address, dirs, files in folder:
    for file in files:
        filenames.append(file)

Выводим список всех файлов, содержащих диалоги

In [15]:
#filenames

In [16]:
new = []
for filename in filenames:
    if filename[0].isnumeric():
        new.append(filename)
print(new)
print(len(new))

['10.txt', '11.txt', '12.txt', '13.06.2018.txt', '13.txt', '13_02_2018.txt', '14.txt', '15.txt', '16.txt', '18.txt', '19.txt', '20.txt', '21.cha', '22.cha', '22_02_2018.cha', '23.cha', '24.cha', '24_03_2018.cha', '25.cha', '26.cha', '27.01.2018.cha', '27.cha', '6_02_2018.txt', '7.txt', '8.txt']
25


In [17]:
paths = []  # список готовых директорий к файлам с диалогами
for filename in new:
    directory = 'childes_Yasha/'
    path = directory + filename
    paths.append(path)
    #print(path)

#print(len(paths))

## Разделение реплик на то, что ребенок слышит, и то, что он говорит

In [18]:
morph = pymorphy2.MorphAnalyzer()

dict_words = []

textChild = ''
textParents = ''

for filename in paths:
    with open(filename, 'r', encoding='utf-8') as f:
        file = f.readlines()
        for line in file:
            line = translit(line, 'ru')
            line = re.sub('йо', 'ё', line)
            line = re.sub('шч', 'щ', line)
            if line.startswith('*ЦХИ'):  # выбираем только реплики ребенка 
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textChild += line.lower()
            elif line.startswith('*'):
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textParents += line.lower()
        
with open('YashaSays.txt', 'w+', encoding='utf-8') as f:
    f.write(textChild)
with open('YashaHears.txt', 'w+', encoding='utf-8') as f:
    f.write(textParents)

### Что Я. говорит

In [22]:
WordsCounting('YashaSays.txt')

Распределение по частям речи  Counter({'NOUN': 3996, 'PRCL': 1876, 'VERB': 1438, 'INTJ': 966, 'ADVB': 911, 'CONJ': 827, 'ADJF': 714, 'PREP': 658, 'NPRO': 486, 'PRED': 434, 'NUMR': 171, 'INFN': 157, 'ADJS': 87, 'GRND': 82, 'PRTS': 59, 'COMP': 11, 'PRTF': 1})
Всего слов в этой части корпуса (вместе с неопределенной частью речи) 14363
Всего слов в этой части корпуса (часть речи определена) 12874
Топ-10 всех слов [('упасть', 147), ('знать', 104), ('папа', 103), ('дом', 97), ('миша', 94), ('ещё', 94), ('мама', 88), ('дать', 78), ('весь', 74), ('есхчё', 74)]
Топ-10 всех существительных [('папа', 103), ('дом', 97), ('миша', 94), ('мама', 88), ('есхчё', 74), ('метр', 71), ('мяч', 67), ('собака', 65), ('поезд', 54), ('яша', 47), ('кубик', 46)]


### Что Я. слышит

In [20]:
WordsCounting('YashaHears.txt')

Распределение по частям речи  Counter({'NOUN': 4799, 'VERB': 3598, 'CONJ': 3335, 'PRCL': 2371, 'ADVB': 2286, 'NPRO': 1998, 'ADJF': 1131, 'PREP': 964, 'INFN': 576, 'INTJ': 512, 'PRED': 500, 'ADJS': 206, 'NUMR': 165, 'GRND': 122, 'COMP': 82, 'PRTS': 60, 'PRTF': 5})
Всего слов в этой части корпуса (вместе с неопределенной частью речи) 23388
Всего слов в этой части корпуса (часть речи определена) 22710
Топ-10 всех слов [('яша', 544), ('давать', 299), ('ещё', 250), ('весь', 145), ('показать', 144), ('буква', 137), ('пожалуйста', 109), ('делать', 109), ('смотреть', 109), ('мочь', 103)]
Топ-10 всех существительных [('яша', 544), ('буква', 137), ('миша', 102), ('поезд', 84), ('дом', 75), ('мама', 70), ('книжка', 66), ('малыш', 51), ('машинка', 51), ('собака', 51)]
